
## Get the data and clean it 

In [ ]:
import numpy as np

import pandas as pd
df=pd.read_html('http://www.infohub.co.ke/2016/04/list-of-nairobi-constituencies-and-wards.html')


In [2]:
df=df[0]

In [3]:

df.head()

,0,1
0,Constituency,Wards
1,Westlands,"Kitisuru , Parklands/Highridge, Kangemi and Mo..."
2,Dagoretti North,"Kilimani, Kawangware, Gatina, Kileleshwa and K..."
3,Dagoretti South,"Mutu-ini/Karen, Ngand'o, Riruta Nyayo Highrise..."
4,Langata,"Karen, Nairobi West, Karura and South C."


In [4]:
df.columns=['constituency','wards']

In [6]:
#For accuracy in the geolocation i append Kenya to the constituencies since some names are same for 
#other world places and i run once and commented the code 
df['constituency'] = df['constituency'].astype(str)+',Kenya'
df.head()

,constituency,wards
0,"Constituency,Kenya",Wards
1,"Westlands,Kenya","Kitisuru , Parklands/Highridge, Kangemi and Mo..."
2,"Dagoretti North,Kenya","Kilimani, Kawangware, Gatina, Kileleshwa and K..."
3,"Dagoretti South,Kenya","Mutu-ini/Karen, Ngand'o, Riruta Nyayo Highrise..."
4,"Langata,Kenya","Karen, Nairobi West, Karura and South C."


In [7]:
df.drop(df.index[0], inplace=True)
df.head()

,constituency,wards
1,"Westlands,Kenya","Kitisuru , Parklands/Highridge, Kangemi and Mo..."
2,"Dagoretti North,Kenya","Kilimani, Kawangware, Gatina, Kileleshwa and K..."
3,"Dagoretti South,Kenya","Mutu-ini/Karen, Ngand'o, Riruta Nyayo Highrise..."
4,"Langata,Kenya","Karen, Nairobi West, Karura and South C."
5,"Kibra,Kenya","Laini Saba, Lindi, Makina, Woodley-Kenyatta Go..."


In [8]:
df.reset_index().head()

,index,constituency,wards
0,1,"Westlands,Kenya","Kitisuru , Parklands/Highridge, Kangemi and Mo..."
1,2,"Dagoretti North,Kenya","Kilimani, Kawangware, Gatina, Kileleshwa and K..."
2,3,"Dagoretti South,Kenya","Mutu-ini/Karen, Ngand'o, Riruta Nyayo Highrise..."
3,4,"Langata,Kenya","Karen, Nairobi West, Karura and South C."
4,5,"Kibra,Kenya","Laini Saba, Lindi, Makina, Woodley-Kenyatta Go..."


In [9]:
df.reset_index(drop=True, inplace=True)
df.head()

,constituency,wards
0,"Westlands,Kenya","Kitisuru , Parklands/Highridge, Kangemi and Mo..."
1,"Dagoretti North,Kenya","Kilimani, Kawangware, Gatina, Kileleshwa and K..."
2,"Dagoretti South,Kenya","Mutu-ini/Karen, Ngand'o, Riruta Nyayo Highrise..."
3,"Langata,Kenya","Karen, Nairobi West, Karura and South C."
4,"Kibra,Kenya","Laini Saba, Lindi, Makina, Woodley-Kenyatta Go..."


### Import the necessary libraries 

In [10]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
print('Libraries imported.')


Libraries imported.


In [114]:
df.iloc[3,0] = "Lang'ata"

### Get the coordinates of all the constituencies

In [157]:
constituency_latlog = {'constituency':[],
                       'latitude':[],
                       'longitude':[]}

geolocator = Nominatim(user_agent="nairobi_explorer")
constituencies = df.constituency.values
for constituency in constituencies:
    try:
        location = geolocator.geocode(constituency)
        latitude = location.latitude
        longitude = location.longitude
        constituency_latlog['constituency'].append(constituency)
        constituency_latlog['latitude'].append(latitude)
        constituency_latlog['longitude'].append(longitude)
#         print('The geograpical coordinate of {} are {}, {}.'.format(constituency, latitude, longitude))
    except:
        pass
#         print('The geograpical coordinate of {} are {}, {}.'.format(constituency, 'None', 'None'))


df1 = pd.DataFrame(constituency_latlog)
df1

,constituency,latitude,longitude
0,"Westlands,Kenya",-1.264726,36.802010
1,"Dagoretti North,Kenya",-1.289675,36.684960
2,"Dagoretti South,Kenya",-1.289675,36.684960
3,Lang'ata,-1.332894,36.781666
4,"Kibra,Kenya",-1.311237,36.786862
5,"Roysambu,Kenya",-1.216342,36.891233
6,"Kasarani,Kenya",-0.714033,36.297753
7,"Ruaraka,Kenya",-1.242994,36.875619
8,"Embakasi South,Kenya",-1.324728,36.887724
9,"Embakasi North,Kenya",-1.324728,36.887724


Some of the locations are not displayed on the map because they are masked by one point for instance embakasi north, south,westis one but others  are miss quoted hence i had to replace with exact coordinates

In [215]:
#Replace the coordinates for westlands, makadara and kasarani constituencies

df1.iloc[6,1]=-1.2254
df1.iloc[6,2]=36.8976
df1.iloc[13,1]=-1.2953
df1.iloc[13,2]=36.8721
df1.iloc[0,1]=-1.2683
df1.iloc[0,2]=36.8111
df1.iloc[16,1]=-1.2619
df1.iloc[16,2]=36.8585

In [216]:
df1

,constituency,latitude,longitude
0,"Westlands,Kenya",-1.268300,36.811100
1,"Dagoretti North,Kenya",-1.289675,36.684960
2,"Dagoretti South,Kenya",-1.289675,36.684960
3,Lang'ata,-1.332894,36.781666
4,"Kibra,Kenya",-1.311237,36.786862
5,"Roysambu,Kenya",-1.216342,36.891233
6,"Kasarani,Kenya",-1.225400,36.897600
7,"Ruaraka,Kenya",-1.242994,36.875619
8,"Embakasi South,Kenya",-1.324728,36.887724
9,"Embakasi North,Kenya",-1.324728,36.887724



Get the geographical location of Nairobi

In [217]:
address = 'Nairobi'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Nairobi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Nairobi are -1.2832533, 36.8172449.


In [246]:
# create map of Nairobi using latitude and longitude values
map_nairobi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df1['latitude'], df1['longitude'], df1['constituency']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nairobi)  
    
map_nairobi

### Next, we are going to start utilizing the Foursquare API to explore the constituencies and segment them.


Now, let's get the top 100 venues that are in Westlandswithin a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.
create the GET request URL.

In [219]:
CLIENT_ID='5RSJPVGGCRLVX5GY2JCDKYKW3K0AOBA1WZ5HO4EWXWDLFGQX'
CLIENT_SECRET='SZR1WWGB1L3WWOKZX1HKH1ETZ4HHAS25YO3ROBW4IQXSREUM'
VERSION=20190321

In [220]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    constituency_latitude, 
    constituency_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c936e409fb6b73b8d777229'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Nairobi',
  'headerFullLocation': 'Nairobi',
  'headerLocationGranularity': 'city',
  'totalResults': 74,
  'suggestedBounds': {'ne': {'lat': -1.2602262954999954,
    'lng': 36.80650269695543},
   'sw': {'lat': -1.2692263045000045, 'lng': 36.79751730304458}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f01e51d02d580f8f1b66ce4',
       'name': 'Fogo Gaucho',
       'location': {'address': 'Waiyaki Way',
        'lat': -1.2641463159681343,
        'lng': 36.80090033911749,
        'labeledLatLngs': [{'label': 'display',
          'lat': -1.2641463159681343,
          

In [221]:


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [222]:
# Clean the json and structure it into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# # filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# # # filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# # # clean columns
# nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Fogo Gaucho,Brazilian Restaurant,-1.264146,36.800900
1,Nyama Mama,African Restaurant,-1.266111,36.802083
2,Brew Bistro & Lounge,Brewery,-1.264832,36.804076
3,Open House,Indian Restaurant,-1.263736,36.802577
4,Urban Eatery,Bistro,-1.265609,36.802212


In [223]:
#Check the number of venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

74 venues were returned by Foursquare.



### Explore Constituencies in Nairobi

Create a function to repeat the same process to all the neighborhoods in Nairobi

In [224]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['constituency', 
                  'latitude', 
                  'longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [225]:
nairobi_venues=getNearbyVenues(names=df1['constituency'],
                                   latitudes=df1['latitude'],
                                   longitudes=df1['longitude']
                                  )

Westlands,Kenya
Dagoretti North,Kenya
Dagoretti South,Kenya
Lang'ata
Kibra,Kenya
Roysambu,Kenya
Kasarani,Kenya
Ruaraka,Kenya
Embakasi South,Kenya
Embakasi North,Kenya
Embakasi Central,Kenya
Embakasi East,Kenya
Embakasi West,Kenya
Makadara,Kenya
Kamukunji,Kenya
Starehe,Kenya
Mathare,Kenya


In [226]:
nairobi_venues.head()

,constituency,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Westlands,Kenya",-1.2683,36.8111,K1 Klub House,-1.268314,36.811406,Nightclub
1,"Westlands,Kenya",-1.2683,36.8111,Parklands Sports Club,-1.268044,36.812612,Gym / Fitness Center
2,"Westlands,Kenya",-1.2683,36.8111,Pitcher And Butch,-1.268284,36.811445,Sports Bar
3,"Westlands,Kenya",-1.2683,36.8111,Villa Rosa Kempinski,-1.271530,36.808799,Hotel
4,"Westlands,Kenya",-1.2683,36.8111,Nairobi Java House,-1.269694,36.809159,Café


In [227]:

nairobi_venues.groupby('constituency').count()



,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
constituency,,,,,,
"Dagoretti North,Kenya",1,1,1,1,1,1
"Dagoretti South,Kenya",1,1,1,1,1,1
"Embakasi Central,Kenya",4,4,4,4,4,4
"Embakasi East,Kenya",4,4,4,4,4,4
"Embakasi North,Kenya",4,4,4,4,4,4
"Embakasi South,Kenya",4,4,4,4,4,4
"Embakasi West,Kenya",4,4,4,4,4,4
"Kamukunji,Kenya",4,4,4,4,4,4
"Kasarani,Kenya",2,2,2,2,2,2


In [228]:
print('There are {} uniques categories.'.format(len(nairobi_venues['Venue Category'].unique())))

There are 40 uniques categories.



### Analyze Each constituency

In [229]:
# one hot encoding
import pandas as pd
nairobi_onehot = pd.get_dummies(nairobi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe

nairobi_onehot['constituency'] = nairobi_venues['constituency'] 


# move neighborhood column to the first column

fixed_columns = [nairobi_onehot.columns[-1]] + list(nairobi_onehot.columns[:-1])

nairobi_onehot = nairobi_onehot[fixed_columns]

nairobi_onehot.head()

,constituency,African Restaurant,Athletics & Sports,BBQ Joint,Baseball Field,Bus Station,Café,Cocktail Bar,Coffee Shop,College Soccer Field,...,Sculpture Garden,Shopping Mall,Soccer Field,Sports Bar,Steakhouse,Trail,Train Station,Video Store,Women's Store,Zoo Exhibit
0,"Westlands,Kenya",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Westlands,Kenya",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Westlands,Kenya",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,"Westlands,Kenya",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Westlands,Kenya",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [230]:
#And let's examine the new dataframe size

nairobi_onehot.shape

(86, 41)


### Next, let's group rows by constituency and by taking the mean of the frequency of occurrence of each category

In [231]:
nairobi_grouped = nairobi_onehot.groupby('constituency').mean().reset_index()
nairobi_grouped

,constituency,African Restaurant,Athletics & Sports,BBQ Joint,Baseball Field,Bus Station,Café,Cocktail Bar,Coffee Shop,College Soccer Field,...,Sculpture Garden,Shopping Mall,Soccer Field,Sports Bar,Steakhouse,Trail,Train Station,Video Store,Women's Store,Zoo Exhibit
0,"Dagoretti North,Kenya",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000
1,"Dagoretti South,Kenya",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000
2,"Embakasi Central,Kenya",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000
3,"Embakasi East,Kenya",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000
4,"Embakasi North,Kenya",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000
5,"Embakasi South,Kenya",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000
6,"Embakasi West,Kenya",0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000
7,"Kamukunji,Kenya",0.50,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000
8,"Kasarani,Kenya",0.50,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000
9,"Kibra,Kenya",0.00,1.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,...,0.0000,0.000000,0.00,0.0000,0.0,0.0000,0.00,0.0000,0.0000,0.0000


In [232]:
#check the shape of the new dataframe
nairobi_grouped.shape

(17, 41)

### Let's print each constituency along with the top 5 most common venues

In [233]:
num_top_venues = 5
for hood in nairobi_grouped['constituency']:
    print("----"+hood+"----")
    temp = nairobi_grouped[nairobi_grouped['constituency'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Dagoretti North,Kenya----
                venue  freq
0         Flea Market   1.0
1  African Restaurant   0.0
2           Hotel Bar   0.0
3              Lounge   0.0
4       National Park   0.0


----Dagoretti South,Kenya----
                venue  freq
0         Flea Market   1.0
1  African Restaurant   0.0
2           Hotel Bar   0.0
3              Lounge   0.0
4       National Park   0.0


----Embakasi Central,Kenya----
                  venue  freq
0                Lounge  0.50
1  Fast Food Restaurant  0.25
2     Convenience Store  0.25
3    African Restaurant  0.00
4         National Park  0.00


----Embakasi East,Kenya----
                  venue  freq
0                Lounge  0.50
1  Fast Food Restaurant  0.25
2     Convenience Store  0.25
3    African Restaurant  0.00
4         National Park  0.00


----Embakasi North,Kenya----
                  venue  freq
0                Lounge  0.50
1  Fast Food Restaurant  0.25
2     Convenience Store  0.25
3    African Restaurant  0.0

In [234]:

#Sort the venues in descending order and put that into a dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Cluster the constituencies

In [249]:
#Run k-means to cluster the constituencies into 5 clusters.
# set number of clusters
kclusters = 5
nairobi_grouped_clustering = nairobi_grouped.drop('constituency', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nairobi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



array([1, 1, 2, 2, 2, 2, 2, 0, 0, 3])

In [252]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each constituency.
# add clustering labels
#constituency_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)

nairobi_merged = nairobi_venues

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

nairobi_merged = nairobi_merged.join(constituency_venues_sorted.set_index('constituency'), on='constituency')
nairobi_merged.head(10) # check the last columns!

,constituency,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Westlands,Kenya",-1.2683,36.8111,K1 Klub House,-1.268314,36.811406,Nightclub,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
1,"Westlands,Kenya",-1.2683,36.8111,Parklands Sports Club,-1.268044,36.812612,Gym / Fitness Center,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
2,"Westlands,Kenya",-1.2683,36.8111,Pitcher And Butch,-1.268284,36.811445,Sports Bar,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
3,"Westlands,Kenya",-1.2683,36.8111,Villa Rosa Kempinski,-1.271530,36.808799,Hotel,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
4,"Westlands,Kenya",-1.2683,36.8111,Nairobi Java House,-1.269694,36.809159,Café,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
5,"Westlands,Kenya",-1.2683,36.8111,Lucca,-1.271505,36.808657,Italian Restaurant,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
6,"Westlands,Kenya",-1.2683,36.8111,Bao Box Cafe,-1.271288,36.810056,Gaming Cafe,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
7,"Westlands,Kenya",-1.2683,36.8111,Cafe Villa Rosa,-1.271484,36.808854,Café,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
8,"Westlands,Kenya",-1.2683,36.8111,La Maison Royale Hotel,-1.265329,36.808667,Hotel,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
9,"Westlands,Kenya",-1.2683,36.8111,Eighty Eight,-1.271572,36.808963,Hotel Bar,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar


### Visualising the clusters 

In [245]:


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nairobi_merged['latitude'], nairobi_merged['longitude'], nairobi_merged['constituency'],nairobi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



### Examine Clusters
Examining each cluster and determining the discriminating venue categories that distinguish each cluster.

Cluster 0: Food outlet

In [239]:
nairobi_merged.loc[nairobi_merged['Cluster Labels'] == 0, nairobi_merged.columns[[1] + list(range(5, nairobi_merged.shape[1]))]]

,latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,-1.268300,36.811406,Nightclub,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
1,-1.268300,36.812612,Gym / Fitness Center,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
2,-1.268300,36.811445,Sports Bar,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
3,-1.268300,36.808799,Hotel,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
4,-1.268300,36.809159,Café,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
5,-1.268300,36.808657,Italian Restaurant,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
6,-1.268300,36.810056,Gaming Cafe,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
7,-1.268300,36.808854,Café,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
8,-1.268300,36.808667,Hotel,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar
9,-1.268300,36.808963,Hotel Bar,0,Hotel,Café,Gaming Cafe,Gastropub,Gym / Fitness Center,Sports Bar,Cocktail Bar,College Soccer Field,Restaurant,Hotel Bar


Cluster 1: Market place

In [240]:
nairobi_merged.loc[nairobi_merged['Cluster Labels'] == 1, nairobi_merged.columns[[1] + list(range(5, nairobi_merged.shape[1]))]]

,latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,-1.289675,36.68665,Flea Market,1,Flea Market,Zoo Exhibit,Women's Store,Gym,Gastropub,Gaming Cafe,Food Court,Fish & Chips Shop,Fast Food Restaurant,Department Store
17,-1.289675,36.68665,Flea Market,1,Flea Market,Zoo Exhibit,Women's Store,Gym,Gastropub,Gaming Cafe,Food Court,Fish & Chips Shop,Fast Food Restaurant,Department Store


Cluster 2: Resting places

In [241]:
nairobi_merged.loc[nairobi_merged['Cluster Labels'] == 2, nairobi_merged.columns[[1] + list(range(5, nairobi_merged.shape[1]))]]

,latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,-1.324728,36.887518,Fast Food Restaurant,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store
54,-1.324728,36.886552,Convenience Store,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store
55,-1.324728,36.884997,Lounge,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store
56,-1.324728,36.883973,Lounge,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store
57,-1.324728,36.887518,Fast Food Restaurant,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store
58,-1.324728,36.886552,Convenience Store,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store
59,-1.324728,36.884997,Lounge,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store
60,-1.324728,36.883973,Lounge,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store
61,-1.324728,36.887518,Fast Food Restaurant,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store
62,-1.324728,36.886552,Convenience Store,2,Lounge,Convenience Store,Fast Food Restaurant,Gym / Fitness Center,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Department Store


Cluster 3: Sporting venues

In [242]:
nairobi_merged.loc[nairobi_merged['Cluster Labels'] == 3, nairobi_merged.columns[[1] + list(range(5, nairobi_merged.shape[1]))]]

,latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,-1.311237,36.786624,Athletics & Sports,3,Athletics & Sports,Zoo Exhibit,Department Store,Gym,Gastropub,Gaming Cafe,Food Court,Flea Market,Fish & Chips Shop,Fast Food Restaurant
